In [1]:
import torch

In [19]:
import numpy as np

In [2]:
torch.__version__

'1.4.0'

In [3]:
DEVICE = torch.device('cuda')

In [4]:
rotation = torch.rand(40, 3, 3, requires_grad=True, device = DEVICE)
translation = torch.rand(40, 3, requires_grad=True , device = DEVICE)
focal_length = torch.rand(1, requires_grad=True, device = DEVICE)
distrotion =  torch.rand(1, requires_grad=True, device = DEVICE)
camera_px = torch.tensor(460.0, requires_grad=False, device = DEVICE)
camera_py = torch.tensor(608.0, requires_grad=False, device = DEVICE)

In [5]:
collect_point2d = []
views_list = []
point3d_list = []
views_count = 0
point3d_count = 0
uv_count = 0
with open('penguin_feature_matching.txt','r') as f:
    views_count, point3d_count, uv_count = f.readline().strip().split(' ')
    collect_point2d = []
    for _ in range(int(uv_count)):
        view, p3d, u,v = list(filter(None,f.readline().strip().split(' ')))
        collect_point2d.append([float(u),float(v)])
        views_list.append(int(view))
        point3d_list.append(int(p3d))

In [6]:
point3d = torch.rand(int(point3d_count), 3, requires_grad=True, device = DEVICE)
point2d = torch.tensor(collect_point2d,requires_grad=False, device = DEVICE)
point3d_index = torch.tensor(point3d_list,dtype=torch.long,requires_grad=False, device = DEVICE)
extrinsic_index = torch.tensor(views_list,dtype=torch.long,requires_grad=False, device = DEVICE)
intrinsic = torch.zeros(3,3, requires_grad=False, device = DEVICE)
extrinsic_pad = torch.zeros(int(uv_count),4,4, requires_grad=False, device = DEVICE)
point3d_pad = torch.ones(int(uv_count),4,1, requires_grad=False, device = DEVICE)

In [7]:
#optimizer = torch.optim.Adam([rotation, translation, focal_length,distrotion,point3d], lr=0.01)
# use per parameter learning rate
optimizer = torch.optim.Adam([
    {'params':focal_length, 'lr':1},
    {'params':rotation},
    {'params':translation},
    {'params':distrotion},
    {'params':point3d},
], lr=0.01) 

In [8]:
for i in range(1000):
    optimizer.zero_grad()
    # build intrisic matrix
    intrinsic[0,0] = focal_length
    intrinsic[1,1] = focal_length
    intrinsic[0,1] = distrotion
    intrinsic[0,2] = camera_px
    intrinsic[1,2] = camera_py
    intrinsic[2,2] = 1.0
    # build extrinsic matrix
    extrinsic_pad[:,:,3] = 1
    extrinsic_pad[:,:3,:3] = rotation[extrinsic_index]
    extrinsic_pad[:,3,:3] = translation[extrinsic_index]
    # build point3d matrix
    point3d_pad[:,:3,0] = point3d[point3d_index]
    # Do projection
    extrinsic_pad_point = torch.matmul(extrinsic_pad,point3d_pad)
    extrinsic_pad_point = extrinsic_pad_point[:,:3,:]
    projected_point = torch.matmul(intrinsic,extrinsic_pad_point[:,:3,:])
    projected_point = projected_point[:,:,0]
    uv_map = torch.empty(int(uv_count),2, device = DEVICE)
    uv_map[:,0] = projected_point[:,0] / projected_point[:,2]
    uv_map[:,1] = projected_point[:,1] / projected_point[:,2]
    # compare after projection with point2d
    uv_difference = torch.pow(point2d - uv_map, 2)
    total_loss = uv_difference.sum()
    total_loss.backward(retain_graph=True)
    optimizer.step()
    if i % 10 == 0:
        print("EPOCH %4d - loss %10.6f - focal %10.6f" % (i,total_loss.item(),focal_length.item()))

EPOCH    0 - loss 1090407168.000000
EPOCH   10 - loss 1038898176.000000
EPOCH   20 - loss 972735488.000000
EPOCH   30 - loss 883624832.000000
EPOCH   40 - loss 776558336.000000
EPOCH   50 - loss 669870272.000000
EPOCH   60 - loss 563143936.000000
EPOCH   70 - loss 468423680.000000
EPOCH   80 - loss 389291712.000000
EPOCH   90 - loss 314161824.000000
EPOCH  100 - loss 240684768.000000
EPOCH  110 - loss 175908448.000000
EPOCH  120 - loss 125914752.000000
EPOCH  130 - loss 92033072.000000
EPOCH  140 - loss 69460400.000000
EPOCH  150 - loss 54170392.000000
EPOCH  160 - loss 43779144.000000
EPOCH  170 - loss 36470292.000000
EPOCH  180 - loss 31077284.000000
EPOCH  190 - loss 26833256.000000
EPOCH  200 - loss 23399028.000000
EPOCH  210 - loss 21120402.000000
EPOCH  220 - loss 19087112.000000
EPOCH  230 - loss 17611740.000000
EPOCH  240 - loss 16395731.000000
EPOCH  250 - loss 15370734.000000
EPOCH  260 - loss 14490982.000000
EPOCH  270 - loss 13726602.000000
EPOCH  280 - loss 13051492.000000

KeyboardInterrupt: 

In [22]:
np.save('instrisic.npy',intrinsic.cpu().detach().numpy())
np.save('rotation.npy',rotation.cpu().detach().numpy())
np.save('translation.npy',translation.cpu().detach().numpy())
np.save('point3d.npy',point3d.cpu().detach().numpy())